# This script quantifies for the association between each LR and the four subtypes
### For one sample 64630

In [1]:
import numpy as np
import csv
import pickle
import matplotlib
import math
import pandas as pd
import matplotlib

In [2]:
def readCsv(x):
  """Parse file."""
  #colNames = ["method", "benchmark", "start", "end", "time", "memory"]
  df = pd.read_csv(x, sep=",")

  return df

def preprocessDf(df):
  """Transform ligand and receptor columns."""
  df["ligand-receptor"] = df["ligand"] + '-' + df["receptor"]
  df["component"] = df["component"] #.astype(str).str.zfill(2)

  return df

In [4]:
# Load gene_ids
gene_ids = []
with open("/Users/victoriagao/local_docs/NEST/stored_variables/gene_ids.txt", 'r') as file:
    for line in file:
        # Remove trailing newline characters and any leading/trailing whitespaces
        line = line.strip()
        gene_ids.append(line)

# Load coordinates
coordinates = np.load("/Users/victoriagao/local_docs/NEST/stored_variables/coordinates.npy")

# Load cell_barcode
with open('/Users/victoriagao/local_docs/NEST/stored_variables/cell_barcode.pkl', 'rb') as file:
    cell_barcode = pickle.load(file)



# # Load pathologist's label
# data_name = 'PDAC_64630'
# if data_name == 'PDAC_64630':
#     pathologist_label_file='/Users/victoriagao/local_docs/NEST/input/pathologist_annot/IX_annotation_artifacts_PDAC64630.csv' #IX_annotation_artifacts.csv' #
#     pathologist_label=[]
#     with open(pathologist_label_file) as file:
#         csv_file = csv.reader(file, delimiter=",")
#         for line in csv_file:
#             pathologist_label.append(line)	
    	
#     barcode_type=dict() # record the type (annotation) of each spot (barcode)
#     for i in range (1, len(pathologist_label)):
#         barcode_type[pathologist_label[i][0]] = pathologist_label[i][1]

# Load subtype label
# subtype_label_file='/Users/victoriagao/local_docs/schwartz_data/PDAC_64630_subtype.csv'
subtype_label_file='/Users/victoriagao/Documents/MSc/Schwartz_lab/experiment_data/Deisha/exp1_C1/cell_type_prediction.csv'
subtype_label=[]
with open(subtype_label_file) as file:
    csv_file = csv.reader(file, delimiter=",")
    for line in csv_file:
        subtype_label.append(line)

barcode_subtype=dict()
for i in range(1,len(subtype_label)):
    barcode_subtype[subtype_label[i][0]]= subtype_label[i][1]

# Load NEST output file into a 2D array
# filename_str = 'NEST_combined_output_PDAC_64630'+'.csv'
# inputFile = '/Users/victoriagao/local_docs/NEST/output/From_Fatema/'+filename_str
# df = pd.read_csv(inputFile, sep=",")
# csv_record_final = df.values.tolist()
# df_column_names = list(df.columns)
# csv_record_final = [df_column_names] + csv_record_final

# Load NEST output 
# df = readCsv("/Users/victoriagao/local_docs/NEST/output/From_Fatema/NEST_combined_output_PDAC_64630.csv")
df = readCsv("/Users/victoriagao/local_docs/NEST/output/From_Fatema/exp1_C1_NEST_combined_rank_product_output_PDAC_140694_top20percent.csv")
df_processed = preprocessDf(df)


In [5]:
df_processed

,from_cell,to_cell,ligand,receptor,attention_score,component,from_id,to_id,ligand-receptor
0,TCAAATTTGAGACTCA-1,TAAGGCCCGTCACCCT-1,CELSR1,HLA,0.972451,-1,1852,1704,CELSR1-HLA
1,TCAAATTTGAGACTCA-1,GAACCCTCTGTGTTCT-1,CELSR1,HLA,0.973245,-1,1852,1152,CELSR1-HLA
2,AATGTGCCCGAGGTGT-1,CCGTATCTCGTCGTAG-1,CELSR1,HLA,0.974442,-1,127,808,CELSR1-HLA
3,CTATTTGGTTACGGAT-1,CACGTTCGTGCTCTAG-1,LGALS3,ITGB4,0.977116,-1,1030,623,LGALS3-ITGB4
4,GCCGAAATTCCTACGT-1,GAAGTTTCCACTCAAT-1,PTPRF,RACK1,0.973938,-1,1321,1166,PTPRF-RACK1
...,...,...,...,...,...,...,...,...,...
444479,ACCCGGATGACGCATC-1,GCGCAAGAGCGCGCTG-1,FN1,SDC1,0.002500,-1,191,1345,FN1-SDC1
444480,TACTTGTTAGTAGTCC-1,AGAAGAGCGCCGTTCC-1,ANXA1,ERBB2,0.009330,-1,1763,277,ANXA1-ERBB2
444481,GCATCGGCCGTGTAGG-1,TTCTTATCCGCTGGGT-1,ANXA1,VCAM1,0.009276,-1,1307,2229,ANXA1-VCAM1
444482,CACTTCGCCACAGGCT-1,AGACCATGGGATACAA-1,SEMA3C,PLXND1,0.009465,-1,635,284,SEMA3C-PLXND1


In [19]:
# Load barcode info
# with open("/Users/victoriagao/local_docs/NEST/stored_variables/PDAC_connected_subtypes/PDAC_64630_combined_barcode_list.pkl", 'rb') as file:
#     combined_barcode_list = pickle.load(file)


In [14]:
LR_pairs = df_processed['ligand-receptor'].unique()

In [125]:
len(LR_pairs)

147

In [15]:
#### i.e. making a list of Dictionary to store matched records for each region
matched_records_set_lib = {}  

for set_index, connected_comp_set in enumerate(filtered_connected_components):
    matched_records = []

    for record in csv_record_final:
        from_cell, to_cell = record[0], record[1]
        if (from_cell in connected_comp_set) and (to_cell in connected_comp_set):
            matched_records.append(record)
    
    # Store matched records for this set index in the dictionary
    matched_records_set_lib[set_index+1] = matched_records

In [16]:
# Create count matrix for all 144 of LR pairs in each region
num_sets = len(matched_records_set_lib)
num_pairs = len(LR_pairs)
count_matrix = pd.DataFrame(index=range(1, num_sets + 1), columns=LR_pairs)

# Fill the count matrix
for set_number, records in matched_records_set_lib.items():
    for lr_pair in LR_pairs:
        count = sum(1 for record in records if f"{record[2]}-{record[3]}" == lr_pair)
        count_matrix.loc[set_number, lr_pair] = count

# # Print the count matrix
# print(count_matrix)

count_matrix_df = pd.DataFrame(count_matrix)

In [17]:
count_matrix_df

,LGALS3-NPTN,FN1-RPSA,PTPRF-RACK1,INS-HLA,TIMP1-LRP1,THBS2-LRP1,THBS1-SDC1,MIF-CD74,LGALS3-ITGB4,RPS19-RPSA,...,COL18A1-ITGB5,HSPG2-DAG1,PSEN1-DAG1,TGFB2-ACVR1B,ITGB1-DAG1,BMP4-BMPR2,BMP2-ENG,HMGB1-AGTRAP,TGFB1-SDC2,no-ligand-no-receptor
1,2,13,4,0,1,0,3,0,20,3,...,0,0,1,0,1,1,0,0,1,1
2,7,20,15,5,11,0,9,8,27,17,...,0,2,0,1,0,0,0,0,0,0
3,1,2,0,1,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6,2,0,17,0,2,0,0,0,2,1,...,1,0,0,0,0,0,1,2,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
# Shows each of the regions what type they are
map_region_type = {}

for idx, component_set in enumerate(filtered_connected_components, start=1):
    component_types = {component['type'] for barcode in component_set for component in combined_barcode_list if component['barcode'] == barcode}
    # print(f"Connected Component {idx} Type: {component_types}")
    map_region_type[idx] = component_types



In [44]:
map_region_type

{1: {'BasalB'},
 2: {'Mixed'},
 3: {'ClassicB'},
 4: {'Mixed'},
 5: {'BasalB'},
 6: {'ClassicA'},
 7: {'ClassicA'},
 8: {'BasalA'},
 9: {'Mixed'},
 10: {'BasalB'},
 11: {'Mixed'},
 12: {'BasalB'},
 13: {'Mixed'},
 14: {'ClassicB'},
 15: {'ClassicB'},
 16: {'BasalB'},
 17: {'Mixed'},
 18: {'ClassicA'},
 19: {'ClassicB'},
 20: {'ClassicA'},
 21: {'BasalB'},
 22: {'Mixed'},
 23: {'BasalB'},
 24: {'BasalB'},
 25: {'Mixed'},
 26: {'Mixed'},
 27: {'ClassicA'},
 28: {'BasalB'},
 29: {'BasalB'},
 30: {'ClassicB'},
 31: {'ClassicA'},
 32: {'BasalB'},
 33: {'BasalB'},
 34: {'ClassicB'},
 35: {'ClassicA'},
 36: {'BasalB'},
 37: {'ClassicA'},
 38: {'BasalA'},
 39: {'BasalB'},
 40: {'BasalB'},
 41: {'BasalB'},
 42: {'Mixed'},
 43: {'BasalB'},
 44: {'ClassicB'},
 45: {'ClassicA'},
 46: {'ClassicB'},
 47: {'BasalB'},
 48: {'BasalB'},
 49: {'BasalA'},
 50: {'BasalB'},
 51: {'ClassicB'},
 52: {'BasalA'},
 53: {'BasalA'}}

## Feature matrix prep

#### Version 1, might be wrong

In [66]:
# Make count matrix for LR vs subtypes
df_reset = count_matrix_df.reset_index().rename(columns={'index': 'Region'})

melted_df = df_reset.melt(id_vars=['Region'], var_name='LR_Pair', value_name='Count')

melted_df['Subtype'] = melted_df['Region'].map(map_region_type)
melted_df['Subtype'] = melted_df['Subtype'].apply(lambda x: next(iter(x)))

grouped_df = melted_df.groupby(['LR_Pair', 'Subtype'])['Count'].sum().reset_index()
# pivot_df = grouped_df.pivot(index='LR_Pair', columns='Subtype', values='Count').fillna(0)
pivot_df = grouped_df.pivot(index='Subtype', columns='LR_Pair', values='Count').fillna(0)

pivot_df


LR_Pair,A2M-TNFRSF14,ADAM9-ITGA6,ADAM9-ITGB5,AIMP1-RACK1,APOE-SDC1,APOE-SDC4,APP-SDC1,ARF1-LRP5,BMP2-ACVR1,BMP2-ENG,...,TIMP1-LRP1,TNC-ITGB6,TNC-SDC1,TNFSF13-TNFRSF14,TNFSF13-TNFRSF1A,TTR-DDR1,TTR-STRA6,VWF-ITGB5,WNT5A-TFRC,no-ligand-no-receptor
Subtype,,,,,,,,,,,,,,,,,,,,,
BasalA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BasalB,0,2,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,1
ClassicA,1,0,0,0,0,0,1,3,1,1,...,2,2,1,1,2,0,0,0,0,0
ClassicB,0,0,0,0,0,0,0,0,0,0,...,3,0,0,0,0,0,1,0,0,0
Mixed,0,0,0,1,6,6,0,0,0,0,...,11,0,0,0,0,5,0,0,1,0


#### Version 2, turning very spot into a sample/observation

In [5]:
# Processing the dataframe to get the counts
df_long = pd.concat([df_processed[['from_cell', 'ligand-receptor']], df_processed[['to_cell', 'ligand-receptor']].rename(columns={'to_cell': 'from_cell'})])
df_counts = df_long.groupby(['from_cell', 'ligand-receptor']).size().unstack(fill_value=0)

# Creating X matrix
X = df_counts

In [6]:
df_counts

ligand-receptor,A2M-TNFRSF14,ADAM9-ITGA6,ADAM9-ITGB5,AIMP1-RACK1,APOE-SDC1,APOE-SDC4,APP-SDC1,ARF1-LRP5,BMP2-ACVR1,BMP2-ENG,...,TIMP1-LRP1,TNC-ITGB6,TNC-SDC1,TNFSF13-TNFRSF14,TNFSF13-TNFRSF1A,TTR-DDR1,TTR-STRA6,VWF-ITGB5,WNT5A-TFRC,no-ligand-no-receptor
from_cell,,,,,,,,,,,,,,,,,,,,,
AAACCGGGTAGGTACC-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
AAAGGCTCTCGCGCCG-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAAGGGATGTAGCAAG-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
AAAGTAGCATTGCTCA-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAAGTGTGATTTATCT-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGTGGTAGGAGGGAT-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TTGTGTATGCCACCAA-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TTGTGTTTCCCGAAAG-1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [159]:
# Creating y vector
# y = pd.Series(barcode_subtype).reindex(X.index)

# # Check for any NaN values in y and fill or remove them
# y = y.fillna('Unknown')  # or y.dropna() if you want to remove barcodes with unknown subtypes


barcodes_in_X = df_counts.index.tolist()
y = pd.Series(barcodes_in_X).map(barcode_subtype)

In [267]:
y

0        BasalB
1      ClassicB
2         Mixed
3         Mixed
4        BasalB
         ...   
716      BasalB
717    ClassicB
718    ClassicB
719       Mixed
720      BasalB
Length: 721, dtype: object

## sklearn.LogisticRegression

In [224]:
from numpy import mean
from numpy import std
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import classification_report
label_encoder = LabelEncoder()
from collections import Counter
# Y = label_encoder.fit_transform(y.index)

# X = X.values

In [172]:
X

array([[0, 0, 0, ..., 0, 0, 2],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [175]:
print(X.shape, y.shape)
print(Counter(y))

(721, 147) (721,)
Counter({'Mixed': 375, 'BasalB': 205, 'ClassicA': 79, 'ClassicB': 49, 'BasalA': 13})


In [246]:
# define the multinomial logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
model.fit(X, y)
model

LogisticRegression(multi_class='multinomial')

In [268]:
# get the mapping of class labels to encoded numbers
class_labels = model.classes_

# display the mapping
for index, label in enumerate(class_labels):
    print(f"Class '{label}' is encoded as {index}.")

Class 'BasalA' is encoded as 0.
Class 'BasalB' is encoded as 1.
Class 'ClassicA' is encoded as 2.
Class 'ClassicB' is encoded as 3.
Class 'Mixed' is encoded as 4.


In [247]:
# model.decision_function(X).shape
# define the model evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=1)
# evaluate the model and collect the scores
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
print('Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Mean Accuracy: 0.558 (0.027)


### Examining the model coefficients/weights

In [219]:
weights = model.coef_
print("Feature Weights:\n", weights)
weights.shape


Feature Weights:
 [[-2.61343772e-02 -7.37870974e-02 -4.32912092e-02 -8.74031202e-02
  -1.12416056e-01 -1.54629889e-01 -8.94104574e-03 -4.94248869e-02
  -2.95114967e-02 -4.92924387e-02 -2.55585992e-02 -8.73846333e-03
  -4.05428495e-02 -1.97676583e-02 -1.45904204e-02 -5.22168606e-02
  -1.47895384e-02 -4.68315218e-02 -2.58605763e-02 -1.16289488e-02
  -5.41878355e-02 -1.43696448e-02 -2.58605763e-02 -1.83124845e-03
  -1.83124845e-03 -4.88890536e-02 -9.09487104e-02 -2.35014355e-02
  -1.17000531e-01 -1.55756861e-02 -6.55034250e-02 -6.03703466e-05
  -2.89907350e-02 -5.07368713e-02 -1.48732962e-01 -6.61916896e-02
  -5.03754914e-02 -2.69878772e-01 -1.34079110e-02 -2.60040394e-02
  -3.86599721e-02 -1.45424613e-01 -4.18498727e-02 -4.19196789e-02
  -3.62053492e-02 -1.59145324e-02 -9.20589615e-02 -5.65943072e-02
  -1.25851173e-01  6.36525926e-01 -1.34706573e-01  2.95730442e-01
  -3.23278841e-01 -1.84513469e-02 -7.94851098e-02 -1.59345622e-02
  -2.58375248e-02 -2.37339545e-02 -2.06140642e-01 -2.86892

(5, 147)

In [245]:
type(weights)

numpy.ndarray

In [185]:
#### Saving the coefficients to table outputs

# Feature names (LR pairs)
feature_names = df_counts.columns

# Class names (assuming model.classes_ gives you the class names in the order they're in the coefficients matrix)
class_names = model.classes_

# Dictionary to hold dataframes for each class
coefficients_dfs = {}

for index, class_name in enumerate(class_names):
    # Create a DataFrame for each class
    class_df = pd.DataFrame(weights[index], index=feature_names, columns=["Coefficient"])
    
    # Sort the DataFrame by the absolute values of the coefficients for better interpretability
    class_df = class_df.reindex(class_df.Coefficient.abs().sort_values(ascending=False).index)
    
    # Save the DataFrame in the dictionary
    coefficients_dfs[class_name] = class_df

    # save to CSV
    class_df.to_csv(f"/Users/victoriagao/local_docs/NEST/stored_variables/coefficients_for_{class_name}.csv")


### Computing p-values for coefficients calculated

In [249]:
## get stand errors for each coefficient
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression

n_iterations = 1000
n_classes = model.coef_.shape[0] 
n_features = model.coef_.shape[1]

# empty array to hold the bootstrap coefficients
bootstrap_coefs = np.zeros((n_iterations, n_classes, n_features))

# bootstrap
for i in range(n_iterations):
    X_sample, y_sample = resample(X, y)
    model.fit(X_sample, y_sample)
    bootstrap_coefs[i] = model.coef_

# standard errors for each coefficient
bootstrap_standard_errors = np.std(bootstrap_coefs, axis=0)

# standard errors for each coefficient across classes
bootstrap_standard_errors_per_feature = np.std(bootstrap_coefs, axis=(0, 1))

print("Bootstrap Standard Errors (per class and feature):", bootstrap_standard_errors)
print("Bootstrap Standard Errors (per feature across all classes):", bootstrap_standard_errors_per_feature)


Bootstrap Standard Errors (per class and feature): [[1.58523732e-02 4.39159692e-02 4.63130995e-02 3.93721659e-02
  6.10322520e-02 4.87421480e-02 7.91466721e-03 2.50941800e-02
  2.03375250e-02 3.12961367e-02 1.86554944e-02 1.28110509e-02
  4.11874095e-02 2.73959680e-02 2.93768222e-02 3.64746753e-02
  1.29313659e-02 4.49431704e-02 2.06436374e-02 7.95753522e-03
  2.52111665e-02 1.19491302e-02 2.03116628e-02 3.59760530e-03
  3.59760530e-03 4.88626276e-02 4.31251111e-02 1.20055503e-02
  7.15330192e-02 1.29326180e-02 3.09497247e-02 2.68748053e-04
  1.94302617e-02 2.12832940e-02 6.07057465e-02 3.33957071e-02
  3.38756897e-02 7.95101109e-02 2.32477046e-02 1.75511354e-02
  3.10900609e-02 5.65954872e-02 2.00002247e-02 2.75078616e-02
  1.95998855e-02 1.40502131e-02 3.00414896e-02 2.68691913e-02
  6.21798837e-02 5.69653280e-01 9.55697088e-02 2.95253777e-01
  1.20390760e-01 1.66723888e-02 7.34843257e-02 1.26372434e-02
  1.67842410e-02 1.39538279e-02 5.64754844e-02 2.77145223e-02
  2.32648555e-02 9.

In [254]:
# weights.shape
bootstrap_standard_errors.shape
# bootstrap_standard_errors_per_feature.shape

(5, 147)

In [256]:
# wald statistics for p-values
from scipy.stats import chi2

wald_stats = (weights / bootstrap_standard_errors) ** 2
p_values = [1 - chi2.cdf(w, 1) for w in wald_stats]

for i, p in enumerate(p_values):
    print(f'Coefficient {i}: p-value = {p}')

Coefficient 0: p-value = [9.92276066e-02 9.29206859e-02 3.49916714e-01 2.64240921e-02
 6.54879780e-02 1.51181424e-03 2.58610849e-01 4.88870199e-02
 1.46755929e-01 1.15248897e-01 1.70677333e-01 4.95173473e-01
 3.24943179e-01 4.70568999e-01 6.19425763e-01 1.52260564e-01
 2.52750152e-01 2.97404063e-01 2.10309762e-01 1.43912358e-01
 3.16059911e-02 2.29143432e-01 2.02951147e-01 6.10739102e-01
 6.10739102e-01 3.17048852e-01 3.49488830e-02 5.02831276e-02
 1.01920038e-01 2.28445737e-01 3.43068823e-02 8.22262865e-01
 1.35688560e-01 1.71310729e-02 1.42830839e-02 4.74745551e-02
 1.36996562e-01 6.88118521e-04 5.64114238e-01 1.38442556e-01
 2.13689784e-01 1.01832363e-02 3.63964812e-02 1.27529619e-01
 6.47149014e-02 2.57344564e-01 2.18111426e-03 3.51791749e-02
 4.29716602e-02 2.63826823e-01 1.58684069e-01 3.16529852e-01
 7.24763756e-03 2.68423286e-01 2.79403306e-01 2.07337408e-01
 1.23708647e-01 8.89632666e-02 2.62147702e-04 3.00589683e-01
 1.59039015e-02 3.70627326e-01 5.44465063e-03 3.47545314e-01

In [266]:
for i, p in enumerate(p_values):
    # Create a DataFrame for each class
    p_value_df = pd.DataFrame(p)

    # save to CSV
    p_value_df.to_csv(f"/Users/victoriagao/local_docs/NEST/stored_variables/pvalues_for_subtype{i}_coefficient.csv")



#### Ignore

In [228]:
# dot_array = np.dot(X.T, X)
# dot_df = pd.DataFrame(dot_array)
# dot_df

,0,1,2,3,4,5,6,7,8,9,...,137,138,139,140,141,142,143,144,145,146
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,8,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,40,6,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,0,1,0,0,2,0,0,0,0,0,...,0,0,0,0,0,42,0,1,0,0
143,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,2,0,0,0
144,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,2,0,0
145,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0


In [238]:
invertible_matrix = np.array([[1, 9], [1, 0]])
np.linalg.inv(invertible_matrix)


array([[ 0.        ,  1.        ],
       [ 0.11111111, -0.11111111]])

In [217]:
# Trying a solution from stack overflow ####


# Trying a solution from stack overflow ####

ModuleNotFoundError: No module named 'regressors'

In [192]:
from scipy import stats

# Assume 'model' is your trained LogisticRegression model

# Calculate the standard errors of the coefficients
# The diagonal of the covariance matrix of the parameter estimates gives the variance
# of the parameter estimates. The standard error is the square root of this variance.
covariance_matrix = np.linalg.inv(np.dot(X.T, X))  # Assuming X is your feature matrix


LinAlgError: Singular matrix

In [ ]:
standard_errors = np.sqrt(np.diag(covariance_matrix))

# Calculate z-scores for each coefficient
z_scores = model.coef_[0] / standard_errors

# Calculate p-values for each coefficient
p_values = [stats.norm.sf(abs(z)) * 2 for z in z_scores]  # two-sided p-value for z-test

# Create a DataFrame for coefficients and p-values
coeff_p_values = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': model.coef_[0],
    'Z-score': z_scores,
    'P-value': p_values
})

print(coeff_p_values)

## Statsmodels

In [186]:
# Number of classes
n_classes = weights.shape[0]
n_classes

5

In [239]:
import statsmodels.api as sm

# Assuming X and Y are your features and target variable
# Add a constant to the feature matrix
X_with_const = sm.add_constant(X)

# Create and fit the MNLogit model
model = sm.MNLogit(y, X_with_const,missing="drop")
result = model.fit(method="lbfgs",maxiter=1000)
# result = model.fit_regularized()

params = result.params
# result.summary()

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          592     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.60944D+00    |proj g|=  3.20111D-01

At iterate    1    f=  1.29474D+00    |proj g|=  1.31522D-01

At iterate    2    f=  1.13466D+00    |proj g|=  3.96773D-02

At iterate    3    f=  1.06931D+00    |proj g|=  2.56639D-02

At iterate    4    f=  1.01058D+00    |proj g|=  9.16841D-02

At iterate    5    f=  9.63739D-01    |proj g|=  1.98402D-02

At iterate    6    f=  9.50457D-01    |proj g|=  1.97600D-02

At iterate    7    f=  9.13365D-01    |proj g|=  1.38113D-02

At iterate    8    f=  8.86782D-01    |proj g|=  1.61766D-02

At iterate    9    f=  8.63581D-01    |proj g|=  1.28221D-02

At iterate   10    f=  8.49072D-01    |proj g|=  8.63292D-03

At iterate   11    f=  8.32272D-01    |proj g|=  1.15237D-02

At iterate   12    f=  8.11900D-01    |proj g|=  1.79539D-02

At iterate   13    f=  7.9

 This problem is unconstrained.



At iterate   42    f=  6.91268D-01    |proj g|=  2.21301D-03

At iterate   43    f=  6.90753D-01    |proj g|=  4.09588D-03

At iterate   44    f=  6.89781D-01    |proj g|=  3.24824D-03

At iterate   45    f=  6.88495D-01    |proj g|=  3.04890D-03

At iterate   46    f=  6.87709D-01    |proj g|=  4.38350D-03

At iterate   47    f=  6.86789D-01    |proj g|=  1.67190D-03

At iterate   48    f=  6.86150D-01    |proj g|=  2.29230D-03

At iterate   49    f=  6.85538D-01    |proj g|=  1.77605D-03

At iterate   50    f=  6.84844D-01    |proj g|=  1.88971D-03

At iterate   51    f=  6.84274D-01    |proj g|=  4.32299D-03

At iterate   52    f=  6.83396D-01    |proj g|=  1.76838D-03

At iterate   53    f=  6.82987D-01    |proj g|=  2.33250D-03

At iterate   54    f=  6.82528D-01    |proj g|=  5.47969D-03

At iterate   55    f=  6.82106D-01    |proj g|=  2.49923D-03

At iterate   56    f=  6.81783D-01    |proj g|=  2.97001D-03

At iterate   57    f=  6.81276D-01    |proj g|=  2.19368D-03

At iter

/Users/victoriagao/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:595: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '


In [240]:
params

,0,1,2,3
const,2.135662,0.149761,0.358144,2.572346
x1,-4.146493,13.405629,-1.059928,-7.590157
x2,6.547295,7.607676,-7.626456,5.848575
x3,-3.685517,13.406478,-16.320878,18.094973
x4,-11.661200,-4.578194,10.424370,8.787397
...,...,...,...,...
x143,7.545453,8.581651,0.420679,7.610445
x144,-6.398582,-1.367382,20.091257,-11.427665
x145,-6.992106,-6.239438,3.124447,11.003024
x146,-7.573849,-1.415771,-1.492178,11.305519


In [242]:
# Get the standard errors
standard_errors = result.bse

# # Compute Wald test statistics and p-values
# wald_stats = (params / standard_errors) ** 2
# p_values = result.pvalues  # statsmodels already provides p-values

# # Print the results
# print(p_values)

ValueError: need covariance of parameters for computing (unnormalized) covariances

In [244]:
result.summary()

ValueError: need covariance of parameters for computing (unnormalized) covariances